In [1]:
import os
import pandas as pd
from pandas import Series, DataFrame
import country_converter as coco

Data sources:
- https://datacatalog.worldbank.org/search/dataset/0037712/World-Development-Indicators

In [2]:
## Get current working directory
print("Current working directory:", os.getcwd())

## Find path to raw data (for the raw files)
raw_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'raw_data'))

## Find path to folder for saving cleaned csv
cleaned_data_path = os.path.abspath(os.path.join(os.getcwd(), '..', 'cleaned_data'))

Current working directory: /Users/jennagreene/Documents/GitHub/HATCH_data/reading_files


In [3]:
target_file = '/WDIEXCEL.xlsx'
target_filepath = raw_data_path + target_file

In [4]:
#df = pd.read_excel(target_filepath)
xlsx = pd.ExcelFile(target_filepath)
df = pd.read_excel(xlsx, 'Data')
df

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,16.433999,16.789043,17.196986,17.597176,18.034249,18.345878,18.695306,19.149942,19.501837,NaN
1,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,6.196543,6.397917,6.580066,6.786218,6.941323,7.096843,7.254828,7.460783,7.599289,NaN
2,Africa Eastern and Southern,AFE,Access to clean fuels and technologies for coo...,EG.CFT.ACCS.UR.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,37.434876,37.660864,37.857526,38.204173,38.303515,38.421813,38.482409,38.692053,38.793983,NaN
3,Africa Eastern and Southern,AFE,Access to electricity (% of population),EG.ELC.ACCS.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,31.682318,31.610692,31.824950,33.744405,38.733352,40.092163,42.880977,44.073912,45.609604,NaN
4,Africa Eastern and Southern,AFE,"Access to electricity, rural (% of rural popul...",EG.ELC.ACCS.RU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,19.307850,18.535523,17.485006,16.329765,24.372504,25.153292,27.227391,29.383000,30.163364,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
383567,Zimbabwe,ZWE,Women who believe a husband is justified in be...,SG.VAW.REFU.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,14.500000,NaN,NaN,NaN,NaN,NaN,NaN
383568,Zimbabwe,ZWE,Women who were first married by age 15 (% of w...,SP.M15.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.700000,NaN,NaN,NaN,5.400000,NaN,NaN
383569,Zimbabwe,ZWE,Women who were first married by age 18 (% of w...,SP.M18.2024.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,32.400000,NaN,NaN,NaN,33.700000,NaN,NaN
383570,Zimbabwe,ZWE,Women's share of population ages 15+ living wi...,SH.DYN.AIDS.FE.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,58.900000,59.100000,59.400000,59.500000,59.700000,59.900000,60.100000,60.300000,60.500000,60.7


In [5]:
def read_wb(df):
    df.drop(columns='Indicator Code', inplace=True)
    df['Data Source'] = 'World Bank'
    df['Unit'] = 'metric tons'
    df['Metric'] = 'Annual production'
    for x in df['Indicator Name']:
        if x=='Aquaculture production (metric tons)':
            df.replace(x, 'Aquaculture Production', inplace=True)
        elif x=='Capture fisheries production (metric tons)':
            df.replace(x, 'Capture Fisheries', inplace=True)
    df.rename(columns={'Indicator Name': 'Technology Name'}, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df = df.iloc[48:]
    df.reset_index(drop=True, inplace=True)
    df.drop(40, inplace=True)
    iso = []
    for country in df['Country Name']:
        if country == 'World':
            iso2 = 'World'
        else:
            iso2 = coco.convert(names=country, to='iso2')
        iso.append(iso2)
    df.drop(columns='Country Code', inplace=True)
    df['Country Code'] = iso
    type_list = []
    for x in range(217):
        name = df['Country Name'].iloc[x]
        if name != 'World':
            type_list.append('National')
        else:
            type_list.append('Global')
    df['Spatial Scale'] = type_list
    df['ID'] = df['Technology Name'] + '_' + df['Metric'] + '_' + df['Country Code']
    omit = []
    for col in df.columns:
        if len(col) != 4 or col == 'Unit':
            omit.append(col)
    empty_rows = df.drop(columns=omit)
    empty_rows.dropna(how='all', inplace=True)
    na_idx = []
    for country in df.index:
        if country not in empty_rows.index:
            na_idx.append(country)
    df.drop(na_idx, inplace=True)
    return df

In [6]:
aqua = df[df['Indicator Name']=='Aquaculture production (metric tons)']
aqua = read_wb(aqua)

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='Indicator Code', inplace=True)
/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data Source'] = 'World Bank'
/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [7]:
cap_fish = df[df['Indicator Name']=='Capture fisheries production (metric tons)']
cap_fish = read_wb(cap_fish)

/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns='Indicator Code', inplace=True)
/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Data Source'] = 'World Bank'
/var/folders/cy/xd4mmw610b3cr6tjgv0h8hlr0000gn/T/ipykernel_31442/172142170.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See 

In [8]:
wb = pd.concat([aqua, cap_fish])

In [9]:
output_file = 'wb.csv'
output_file_path = os.path.join(cleaned_data_path, output_file)

wb.to_csv(output_file_path)
print("Data saved to:", output_file_path)


Data saved to: /Users/jennagreene/Documents/GitHub/HATCH_data/cleaned_data/wb.csv
